# Allele-Specific Expression

RNA sequencing can distinguish transcripts expressed from different copies of genes on homologous chromosomes when single-nucleotide polymorphisms (perhaps silent) distinguish the two alleles. Linkage between these distinctive SNPs and _cis_-regulatory sequences can provide information on regulatory variation within a shared cellular context.

### From last time...

The `numpy.random` submodule of the `numpy` module has functions to generate (pseudo-)random numbers. To use this, we'll `import` this submodule and use it like this:

```
import numpy as np
print(np.random.choice(['A', 'C', 'G', 'T'], 8))
print(np.random.choice([0, 1], 8))
```

We can *count* how many of our random 0-or-1 choices came up 1  using the `sum()` function, like this:

```
sum(np.random.choice([0, 1], 8))
```

We can generate 10,000 random samples of 8 reads each, and tabulate the number of 1s in each sample:

First we create an empty counting array with 9 entries for 0 through 8 inclusive
```
allele_counts_8 = [0] * 9
```
Then we'll run a for loop 10,000 times. We don't actually use the loop varible inside the `for` loop, so by convention we'll just call it `_`.
```
for _ in range(0, 10000):
    k = sum(np.random.choice([0, 1], 8))
    allele_counts_8[k] = allele_counts_8[k] + 1
allele_counts_8
```

Instead of random sampling, we can use the **binomial distribution**.

The scipy package contains a statistics module with a sub-module specific for the binomial distribution. We can get the probability
```
P( k successes out of N trials with probability p of success per trial )
```
using
```
binom.pmf(k, N, p)
```
"pmf" here stands for "probability mass function".

For instance, we can ask about exactly 2 "successes" out of 8 "trials" when each trial has a 50-50 chance of "success".

Import the `binom` sub-module of the `scipy.stats`
```
from scipy.stats import binom
```
Then, use the `binom.pmf()` function like
```
binom.pmf(2, 8, 0.5)
```

We can also import the `matplotlib.pyplot` module and plot the binomial distribution.

```
import matplotlib.pyplot as plt
```

We can use the `range()` function to get a list of the numbers from 0 to 8 (**including 8**, so in Python style we have a range from 0 (included) to 9 (excluded)), and then use the whole range as the value for `k` (the number of successes) to get a range of probabilities.

```
range(0,9)
binom.pmf(range(0,9), 8, 0.5)
plt.plot(binom.pmf(range(0, 9), 8, 0.5))
```

We can make the same kind of plot for 32 trials instead of just 8

We can use `sum()` to add up probabilities across a `range()` of different values.

For instance, the total probability needs to add up to 1
```
sum(binom.pmf(range(0,33), 32, 0.5))
```

We can also ask the question, what is the chance that we get *4 or fewer* successes in 32 trials?

We need `k` to run from 0 through 4 inclusive, so we use `range(0,5)` to do:
```
sum(binom.pmf(range(0,5), 32, 0.5))
```

**Exercise**

Compute the probabilities below, for getting *no more than 12.5% (i.e. one in eight)* successes in a varying number of trials:
- *1 or fewer* successes in *8 trials*
- *2 or fewer* successes in *16 trials*
- *8 or fewer* successes in *64 trials*

## Maximum Likelihood Estimation

Instead of simply *testing* whether our observed data could be generated under the binomial model with a 50% chance of success each trial, we want to _estimate_ the chance of success in each trial, given our data.

To do this, we will start by making a graph where we consider all possible values for the probability of success in one trial, and then figure out the likelihood function P( 4 reads out of 32 | bias ). We'll use the `binom.pmf` function again, but now we'll consider many different values for the 3rd _p_ parameter instead of the first _k_ one.

This graph looks similar to others that we've made, but the axes are different -- we'll add x and y axis labels to emphasize this difference.

First make an array of possible _p_ values of 0, 0.01, 0.02, ..., 1.00
```
prob_success = np.arange(0,1,0.01)
```


Then, compute the *likelihood*, that is, the probability of generating the observed data (4 reads out of 32) from the mode (our specified value of *p*). Conveniently, we can run these over the whole array of values at once.
```
binom.pmf(4, 32, prob_success)
```

Now, we can plot these likelihoods, and then use the `plt.xlabel()` and `plt.ylabel()` functions to add axis labels.
```
plt.plot(prob_success, binom.pmf(4, 32, prob_success))
plt.xlabel('P(success) per trial')
plt.ylabel('P(4 success / 32 total)')
```

As we discussed before, we often want to work with log likelihood functions. In fact, `binom` has the built-in ability to give us a log probability that will be numerically stable when the actual likelihood is a very tiny number, using the `logpmf` method.
```
plt.plot(prob_success, binom.logpmf(4, 32, prob_success))
```

In this plot, the likelihood function looks pretty flat around 0.25, but we might want to adjust the y-axis to focus on the region of high likelihood -- the default puts a lot of emphasis on parts of the plot where the likelihood is very small. The `plt.axis(x_min, x_max, y_min, y_max)` function does this: 
```
plt.axis([0.0, 1.0, -10, 0])
```

We need to find the point on the x-axis where the likelihood is maximized. We can probably guess that this will happen at 0.125, but we can use algorithms from Scipy to find the best likelihood. These methods are typically expressed in terms of _minimization_, and so we'll minimize the negative log likelihood which is equivalent to finding the maximum of the likelihood.

To do this, we define a function to compute the negative log likelihood, called `negloglik`, and then use `minimize_scalar` from `scipy.optimize` to find the allele skew value that maximizes the likelihood of our data.

This will import the minimization sub-module
```
from scipy.optimize import minimize_scalar
```
The negative log likelihood function is quite simple:
```
def negloglik(skew):
    return -binom.logpmf(4, 32, skew)
```
And we can then use `minimize_scalar` to find the best skew estimate
```
best = minimize_scalar(negloglik, bounds=(0,1), method='bounded')
```

**Exercise**

To see how the total number of trials, _N_, affects the sharpness of our maximum likelihood estimates, plot the log-likelihood functions for:


we'll plot the log likelihood functions for:
- *1 or fewer* successes in *8 trials*
- *2 or fewer* successes in *16 trials*
- *4 or fewer* successes in *32 trials*
- *8 or fewer* successes in *64 trials*

all on the same graph. Don't forget to set the axis ranges to focus on the part of the graph where the likelihood is highest.